# Dataset Viewer

This notebook loads and visualises the collected dataset of micromagnetic hysteresis simulations for 50 nm single-grain cubes. Each row in the dataset represents one simulation with randomly sampled intrinsic parameters ($M_s$, $A$, $K_1$) and the corresponding computed extrinsic properties ($H_c$, $M_r$, $BH_{\max}$).

In [ ]:
import matplotlib.pyplot as plt
import mammos_entity as me

## Load the dataset

Read the collected CSV file using `mammos_entity`. The file contains ontology-annotated columns for both intrinsic inputs ($M_s$, $A$, $K_1$) and computed extrinsic outputs ($H_c$, $M_r$, $BH_{\max}$).

In [ ]:
collection = me.io.entities_from_file("single_grain_cube_50nm_aligned.csv")
collection

Create a pandas DataFrame for easier handling and visualisation.

In [ ]:
df = collection.to_dataframe(include_units=False)
df

## Data quality overview

Convert to a pandas DataFrame and check for failed simulations (rows containing NaN values). Rows with NaN values correspond to simulations where the magnetisation did not flip within the applied field range (down to $-10$ T), making it impossible to extract the coercivity, remanence, or maximum energy product.

In [ ]:
# Stats
# Total invalid entries (rows with NaN values in any of the columns)
total_invalid_entries = df.isna().any(axis=1).sum()
print(f"Total invalid entries: {total_invalid_entries}")

# Total valid entries (rows without any NaN values)
total_valid_entries = df.dropna().shape[0]
print(f"Total valid entries: {total_valid_entries}")

# Percentage of valid entries
percentage_valid = (total_valid_entries / len(df)) * 100
print(f"Percentage of valid entries: {percentage_valid:.2f}%")


In [ ]:
plt.scatter(df["Ms"], df["K1"], c=df["Mr"])
plt.xlabel("Ms (A/m)")
plt.ylabel("K1 (J/m^3)")
plt.colorbar(label="Mr (A/m)")
plt.title("Scatter plot of Ms vs K1 colored by Mr")

## Visualise the parameter space

Scatter plots of the intrinsic parameter pairs coloured by the extrinsic properties. Red crosses mark simulations where the magnetisation did not reverse (NaN results). Empty regions in the plots correspond to parameter combinations that were rejected during sampling either because they would produce an exchange length or domain wall width smaller than the 1 nm cell size or produce invalid results.

In [ ]:
# create the same plot but with x for all nan values along with all the valid points
nan_df = df[df["Mr"].isna()]
plt.scatter(nan_df["Ms"], nan_df["K1"], marker='x', color='red')
plt.scatter(df["Ms"], df["K1"], c=df["Mr"])
plt.xlabel("Ms (A/m)")
plt.ylabel("K1 (J/m^3)")
plt.title("Scatter plot of Ms vs K1 colored by Mr with NaN values marked")
plt.colorbar(label="Mr (A/m)")

In [ ]:
def plot(color_name):
    fig, ax = plt.subplots(1, 3, figsize=(10, 5))

    color = df[color_name]

    ax[0].scatter(df["Ms"], df["K1"], c=color)
    ax[0].set_xlabel(r"$M_s \, (\mathrm{A/m})$")
    ax[0].set_ylabel(r"$K \, (\mathrm{J/m^3})$")

    ax[1].scatter(df["Ms"], df["A"], c=color)
    ax[1].set_xlabel(r"$M_s \, (\mathrm{A/m})$")
    ax[1].set_ylabel(r"$A \, (\mathrm{J/m})$")

    scatter = ax[2].scatter(df["K1"], df["A"], c=color)
    ax[2].set_xlabel(r"$K \, (\mathrm{J/m^3})$")
    ax[2].set_ylabel(r"$A \, (\mathrm{J/m})$")
    fig.colorbar(scatter, ax=ax[2], label=color_name)
    
    plt.tight_layout()
    plt.show()

## Extrinsic property maps

Three-panel scatter plots showing combinations of intrinsic parameters, coloured by each extrinsic property ($H_c$, $M_r$, $BH_{\max}$) in turn. This reveals which regions of the intrinsic parameter space yield desirable magnetic performance. Empty regions in the plots correspond to parameter combinations that were rejected during sampling either because they would produce an exchange length or domain wall width smaller than the 1 nm cell size or produce invalid results.

**Note on coverage of the intrinsic parameter space:** The parameter space is not uniformly filled because samples where the exchange length $l_A = \sqrt{2A / (\mu_0 M_s^2)}$ or the domain wall width $l_K = \sqrt{A / K_1}$ falls below the mesh cell size (1 nm) are rejected and re-sampled to ensure accurate and stable simulations.

In [ ]:
plot("Hc")

In [ ]:
plot("Mr")

In [ ]:
plot("BHmax")